In [ ]:
import pandas as pd
import gc

In [5]:
# 📥 Cargar datasets
print("Cargando datasets...")
sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')
# Verificación inicial
print(f"Sell-In: {sellin.shape[0]} filas y {sellin.shape[1]} columnas")
print(f"Productos: {productos.shape[0]} filas y {productos.shape[1]} columnas")
print(f"Stocks: {stocks.shape[0]} filas y {stocks.shape[1]} columnas")

Cargando datasets...
Sell-In: 2945818 filas y 7 columnas
Productos: 1251 filas y 6 columnas
Stocks: 13691 filas y 3 columnas


In [6]:
# 📦 Unir datasets en un DataFrame principal
print("Unificando datasets...")
df = (
    sellin
    .merge(productos, how="left", on="product_id")
    .merge(stocks, how="left", on=["product_id", "periodo"])
)
del sellin, productos, stocks
gc.collect()

# 📊 Agrupar ventas efectivas por periodo, cliente y producto
print("Agrupando ventas efectivas...")
ventas_agg = (
    df.groupby(["periodo", "customer_id", "product_id"])
    .agg({
        "tn": "sum",
        "cust_request_tn": "sum",
        "cust_request_qty": "sum",
        "cat1": "first",
        "cat2": "first",
        "cat3": "first",
        "brand": "first",
        "sku_size": "first",
        "stock_final": "first",
        "plan_precios_cuidados": "first"
    })
    .reset_index()
)

# 📅 Calcular vida útil de productos y clientes
print("Calculando vida útil de productos y clientes...")
vida_producto = df.groupby('product_id')['periodo'].agg(['min', 'max']).rename(columns={'min': 'nacimiento_producto', 'max': 'muerte_producto'})
vida_cliente = df.groupby('customer_id')['periodo'].agg(['min', 'max']).rename(columns={'min': 'nacimiento_cliente', 'max': 'muerte_cliente'})

# 📅 Crear combinaciones completas <producto, cliente, periodo>
print("Generando combinaciones completas...")
productos_unicos = df['product_id'].unique()
clientes_unicos = df['customer_id'].unique()
periodos_unicos = df['periodo'].unique()

# Utilizar MultiIndex para crear todas combinaciones posibles
idx = pd.MultiIndex.from_product([productos_unicos, clientes_unicos, periodos_unicos], names=['product_id', 'customer_id', 'periodo'])
completo = idx.to_frame(index=False)

# 📊 Filtrar combinaciones válidas según vida de producto y cliente
print("Filtrando combinaciones válidas...")
completo = (
    completo
    .merge(vida_producto, on='product_id', how='left')
    .merge(vida_cliente, on='customer_id', how='left')
)
completo = completo[
    (completo['periodo'] >= completo['nacimiento_producto']) &
    (completo['periodo'] <= completo['muerte_producto']) &
    (completo['periodo'] >= completo['nacimiento_cliente']) &
    (completo['periodo'] <= completo['muerte_cliente']) &
    (completo['nacimiento_producto'] < 201910)
]

# 📦 Unir con ventas efectivas (rellenar con 0 si no hubo venta)
print("Unificando con ventas efectivas...")
df_completo = completo.merge(ventas_agg, how='left', on=['product_id', 'customer_id', 'periodo'])
df_completo['tn'] = df_completo['tn'].fillna(0)

# 📅 Crear columna periodo_dt y periodo_target (para target futuro)
print("Creando columnas de fechas y target...")
df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'], format='%Y%m')
df_completo['periodo_target_dt'] = df_completo['periodo_dt'] + pd.DateOffset(months=2)

ventas_futuras = df_completo[['product_id', 'customer_id', 'periodo_dt', 'tn']].rename(columns={'periodo_dt': 'periodo_target_dt', 'tn': 'target'})
df_final = df_completo.merge(ventas_futuras, on=['product_id', 'customer_id', 'periodo_target_dt'], how='left').drop(columns=['periodo_target_dt'])

# 🧹 Liberar memoria
del df, ventas_agg, completo, ventas_futuras
gc.collect()

print(f"✅ Dataset final generado con {len(df_final):,} filas.")


Unificando datasets...
Agrupando ventas efectivas...
Calculando vida útil de productos y clientes...
Generando combinaciones completas...
Filtrando combinaciones válidas...
Unificando con ventas efectivas...
Creando columnas de fechas y target...
✅ Dataset final generado con 15,507,462 filas.


In [ ]:
# 📥 Cargar datasets
print("Cargando datasets...")
sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')


df = pd.merge(sellin, productos, how="left", on="product_id")
df = df.merge(stocks, how="left", on=["product_id", "periodo"])
del sellin, productos, stocks

# Agrupar ventas por periodo, cliente y producto
dt = df.groupby(by=["periodo","customer_id","product_id"]).agg({"tn":"sum",
                                                                "cust_request_tn":"sum",
                                                                "cust_request_qty":"sum",
                                                                "cat1":"first",
                                                                "cat2":"first",
                                                                "cat3":"first",
                                                                "brand":"first",
                                                                "sku_size":"first",
                                                                "stock_final":"first",
                                                                "plan_precios_cuidados":"first"
                                                                }).reset_index()
# 1. Calcular primeros períodos

periodo_producto =  df.groupby(['periodo', 'product_id'])["tn"].sum().rename('periodo_producto')
nacimiento_producto = df.groupby('product_id')['periodo'].min().rename('nacimiento_producto')
muerte_cliente = df.groupby('customer_id')['periodo'].max().rename('muerte_cliente')

# 2. Crear todas las combinaciones posibles
productos = df['product_id'].unique()
clientes = df['customer_id'].unique()
periodos = df['periodo'].unique()

# 3. Crear matriz de combinaciones válidas usando broadcasting
idx = pd.MultiIndex.from_product([productos, clientes, periodos], names=['product_id', 'customer_id', 'periodo'])
completo = idx.to_frame(index=False)
# 4 filtrar combinaciones periodo_producto
completo = completo.merge(periodo_producto, on=['periodo', 'product_id'], how='left')
completo = completo[completo['periodo_producto'].notna()]
# 5 Filtrar combinaciones nacimiento_producto
completo = completo.merge(nacimiento_producto, on='product_id', how='left')
completo = completo[completo['periodo'] >= completo['nacimiento_producto']]
completo = completo[completo['nacimiento_producto'] < 201910]

# 6. Filtrar combinaciones muerte_cliente
completo = completo.merge(muerte_cliente, on='customer_id', how='left')
completo = completo[completo['periodo'] <= completo['muerte_cliente']]


# 7. Unir con datos reales
df_completo = completo.merge(dt, how='left')
df_completo['tn'] = df_completo['tn'].fillna(0)

# Convertir periodo a datetime con día fijo
df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'], format='%Y%m')

# Crear columna periodo_target_dt desplazada 2 meses
df_completo['periodo_target_dt'] = df_completo['periodo_dt'] + pd.DateOffset(months=2)

# DataFrame auxiliar con ventas futuras
ventas_futuras = df_completo[['periodo_dt', 'customer_id', 'product_id', 'tn']].copy()
ventas_futuras = ventas_futuras.rename(columns={
    'periodo_dt': 'periodo_target_dt',
    'tn': 'target'
})

# Hacemos el merge usando fechas datetime directamente
dt = pd.merge(
    df_completo,
    ventas_futuras,
    how='left',
    on=['periodo_target_dt', 'customer_id', 'product_id']
)
dt = dt.drop(columns=['periodo_target_dt']) # Ya usamos esta para hacer el merge
del df_completo, ventas_futuras, completo, idx, periodo_producto, nacimiento_producto, muerte_cliente
gc.collect()

dt.shape

Cargando datasets...


(16998193, 18)